# Data Cleansing

In [1]:
import pandas as pd
from io import StringIO
import json

## Read Raw Data

In [2]:
columns = ['video_id', 'title', 'channel_title', 'category_id',
           'tags', 'views', 'likes', 'dislikes', 'comment_total',
           'thumbnail_link', 'date']

gb_videos = pd.read_csv('raw/GBvideos.csv', usecols=columns)
us_videos = pd.read_csv('raw/USvideos.csv', usecols=columns)

gb_videos['country'] = 'GB'
us_videos['country'] = 'US'

videos = pd.concat([gb_videos, us_videos], ignore_index=True)

In [3]:
print(videos['date'].unique())

['13.09' '14.09' '15.09' '16.09' '17.09' '18.09' '19.09' '20.09' '21.09'
 '22.09' '23.09' '24.09' '24.09l7yxJDFvTRM' '25.09' '26.09'
 '26.09t2oVUxTV4WA' '27.09' '28.09' '29.09' '30.09' '01.10' '02.10'
 '03.10' '04.10' '05.10' '06.10' '07.10' '08.10' '09.10' '10.10' '11.10'
 '12.10' '13.10' '14.10' '15.10' '16.10' '17.10' '18.10' '19.10' '20.10'
 '21.10' '22.10' '24.09xcaeyJTx4Co' '26.0903jeumSTSzc' '100']


Observed that there are corrupted observations in column `date`, we need to cleanse them beforehand.

## Re-read Raw Data and Cleanse Date

The followings will be done:
- Observations with date in format dd.mm followed with `video_id`, `\n` will be inserted in the middle
- Observations with date = 100: only happens for date = `09.10` in US dataset, `100` will be replaced by empty string

In [4]:
def cleanse_date(country, old_dates, new_dates):
    with open('raw/{}videos.csv'.format(country), 'r', encoding='utf-8') as f:
        data_string = f.read()
    
    for old_date, new_date in zip(old_dates, new_dates):
        data_string = data_string.replace(old_date, new_date)
    
    return data_string

In [5]:
gb = cleanse_date(
    'GB',
    ['24.09l7yxJDFvTRM', '26.09t2oVUxTV4WA'],
    ['24.09\nl7yxJDFvTRM', '26.09\nt2oVUxTV4WA'])

us = cleanse_date(
    'US',
    ['24.09xcaeyJTx4Co', '26.0903jeumSTSzc', '.jpg,100,09.10'],
    ['24.09\nxcaeyJTx4Co', '26.09\n03jeumSTSzc', '.jpg,09.10'])

We have to force the column type of `date` to be string. Otherwise, it will be imported as float.

In [6]:
gb_videos = pd.read_csv(StringIO(gb), dtype={'date': object}, na_values='[none]')
us_videos = pd.read_csv(StringIO(us), dtype={'date': object}, na_values='[none]')

gb_videos['country'] = 'GB'
us_videos['country'] = 'US'

videos = pd.concat([gb_videos, us_videos], ignore_index=True)

Double check to see if `date` is fixed:

In [7]:
print(videos['date'].unique())

['13.09' '14.09' '15.09' '16.09' '17.09' '18.09' '19.09' '20.09' '21.09'
 '22.09' '23.09' '24.09' '25.09' '26.09' '27.09' '28.09' '29.09' '30.09'
 '01.10' '02.10' '03.10' '04.10' '05.10' '06.10' '07.10' '08.10' '09.10'
 '10.10' '11.10' '12.10' '13.10' '14.10' '15.10' '16.10' '17.10' '18.10'
 '19.10' '20.10' '21.10' '22.10']


In [8]:
# Change date to datetime format
videos['date'] = pd.to_datetime(videos['date'] + '.2017', format='%d.%m.%Y')

## Further Cleansing

Before uploading to S3 again, the followings will be checked:

- Duplicates
- Numebr of views should be monotonic increasing

Any of above found will be removed immediately. Number of likes, dislikes and comments will not be checked as users can revoke their votes or delete their comments for a video.

In [9]:
# Remove duplicates
videos = videos.drop_duplicates().reset_index(drop=True)

In [10]:
# Check increasing
check_increasing = videos.groupby(['video_id', 'country'])['views'].apply(lambda x: x.is_monotonic)
check_increasing = check_increasing.reset_index()
check_increasing.columns = ['video_id', 'country', 'view_increasing']

In [11]:
check_increasing[check_increasing['view_increasing'] == False]

,video_id,country,view_increasing
433,5gH8iXNW8wE,US,False


Since a video with non-monotonic increasing view counts is found, we first take a look on what it looks like:

In [12]:
videos.query('video_id == "5gH8iXNW8wE" and country == "US"')

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,country
14796,5gH8iXNW8wE,Celebrating Selena Quintanilla,googledoodles,10,NaN,657842,57230,1914,0,https://i.ytimg.com/vi/5gH8iXNW8wE/default.jpg,2017-10-17,US
14996,5gH8iXNW8wE,Celebrating Selena Quintanilla,googledoodles,10,NaN,0,0,0,0,https://i.ytimg.com/vi/5gH8iXNW8wE/default.jpg,2017-10-18,US
15205,5gH8iXNW8wE,Celebrating Selena Quintanilla,googledoodles,10,NaN,0,0,0,0,https://i.ytimg.com/vi/5gH8iXNW8wE/default.jpg,2017-10-19,US
15453,5gH8iXNW8wE,Celebrating Selena Quintanilla,googledoodles,10,NaN,0,0,0,0,https://i.ytimg.com/vi/5gH8iXNW8wE/default.jpg,2017-10-20,US
15703,5gH8iXNW8wE,Celebrating Selena Quintanilla,googledoodles,10,NaN,0,0,0,0,https://i.ytimg.com/vi/5gH8iXNW8wE/default.jpg,2017-10-21,US
15930,5gH8iXNW8wE,Celebrating Selena Quintanilla,googledoodles,10,NaN,0,0,0,0,https://i.ytimg.com/vi/5gH8iXNW8wE/default.jpg,2017-10-22,US


After searching this video in Youtube, it seems that views, likes, dislikes and comments are disabled for this video. This video will be removed from analysis.

In [13]:
videos = videos.merge(
    check_increasing,
    on=['video_id', 'country'],
    how='left').query('view_increasing').drop('view_increasing', axis=1).reset_index(drop=True)

## Get Category Name from JSON

In [14]:
def get_category_name(country):
    with open('raw/{}_category_id.json'.format(country), 'r') as f:
        json_cat = json.load(f)
        
    category_name = {'category_id': [], 'category_name': [], 'country': country}
    for item in json_cat['items']:
        category_name['category_id'] += [int(item['id'])]
        category_name['category_name'] += [item['snippet']['title']]
    
    return pd.DataFrame(category_name)

In [15]:
gb_category_names = get_category_name('GB')
us_category_names = get_category_name('US')

category_names = pd.concat([gb_category_names, us_category_names], ignore_index=True)

In [16]:
videos = videos.merge(category_names, on=['category_id', 'country'], how='left')

See if any videos cannot be associated with a category name:

In [17]:
videos[pd.isna(videos['category_name'])]

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,country,category_name
114,A520AWwJ9lQ,XQ Super School Live | Broadcast and Pre-Show,XQ America,29,NaN,104920,268,60,6,https://i.ytimg.com/vi/A520AWwJ9lQ/default.jpg,2017-09-13,GB,NaN
340,A520AWwJ9lQ,XQ Super School Live | Broadcast and Pre-Show,XQ America,29,NaN,114203,270,62,8,https://i.ytimg.com/vi/A520AWwJ9lQ/default.jpg,2017-09-14,GB,NaN
539,k2EIFLJIINA,Sonic Mania - Green Hill Zone Act 1 in 0'28'61...,Jerrel Syrup,29,sonic|mania|speed|run|green|hill|zone|act|time...,11405,52,0,20,https://i.ytimg.com/vi/k2EIFLJIINA/default.jpg,2017-09-15,GB,NaN
584,A520AWwJ9lQ,XQ Super School Live | Broadcast and Pre-Show,XQ America,29,NaN,119431,274,62,8,https://i.ytimg.com/vi/A520AWwJ9lQ/default.jpg,2017-09-15,GB,NaN
782,k2EIFLJIINA,Sonic Mania - Green Hill Zone Act 1 in 0'28'61...,Jerrel Syrup,29,sonic|mania|speed|run|green|hill|zone|act|time...,11637,54,0,19,https://i.ytimg.com/vi/k2EIFLJIINA/default.jpg,2017-09-16,GB,NaN
2688,iPVqwl5dUbw,Denise Welch opens up about losing her mum to ...,Cancer Research UK,29,Cancer Research UK|cancer patient|CR-UK|CRUK|c...,1475,12,0,0,https://i.ytimg.com/vi/iPVqwl5dUbw/default.jpg,2017-09-26,GB,NaN
2963,iPVqwl5dUbw,Denise Welch opens up about losing her mum to ...,Cancer Research UK,29,Cancer Research UK|cancer patient|CR-UK|CRUK|c...,1510,12,0,0,https://i.ytimg.com/vi/iPVqwl5dUbw/default.jpg,2017-09-27,GB,NaN
3564,oFPSsGg6lJI,Manon Carpenter supports Cycling UK's Trails f...,Cycling UK,29,cycling|mountain biking|Trails for Wales|MTB|W...,1441,5,0,0,https://i.ytimg.com/vi/oFPSsGg6lJI/default.jpg,2017-09-30,GB,NaN


There are 4 videos, a total of 8 records which do not have category name. They all have the same Category ID, 29, from UK. They will be removed from analysis as well since they do not account for a large propotion of the data.

In [18]:
videos = videos[~pd.isna(videos['category_name'])].reset_index(drop=True)

## Video Tags

We will also create a dataframe with video id and tags.

In [19]:
videos_with_tags = videos[~pd.isna(videos['tags'])]

In [20]:
video_tags = pd.DataFrame(
    videos_with_tags['tags'].str.split('|').tolist(),
    index=videos_with_tags['video_id']).stack()

In [21]:
video_tags = video_tags.reset_index(['video_id', 0])
video_tags.columns = ['video_id', 'tag']

## Get Latest Date for Each Video by Country

In [22]:
latest_dates = videos.groupby(['video_id', 'country'])['date'].max().reset_index()
latest_dates['latest_date'] = 1

videos = videos.merge(latest_dates, on=['video_id', 'country', 'date'], how='left')
videos['latest_date'].fillna(0, inplace=True)
videos['latest_date'] = videos['latest_date'].astype(int)

## Output Cleansed Data

In [23]:
videos = videos[
    ['video_id', 'title', 'channel_title', 'category_name', 'views',
     'likes', 'dislikes', 'comment_total', 'date', 'country', 'latest_date']]

In [24]:
videos.to_csv('cleansed/videos.csv', index=False)
video_tags.to_csv('cleansed/video_tags.csv', index=False)